In [3]:
import openai
from dotenv import find_dotenv, load_dotenv
import time
import logging
from datetime import datetime
import os

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
client = openai.OpenAI()
model = "gpt-3.5-turbo-16k"

In [13]:
# ==  Create our Assistant (Uncomment this to create your assistant) ==
personal_trainer_assis = client.beta.assistants.create(
    name="個人訓練教練助理",
    instructions="""你是最好的私人教練和營養師，知道如何讓客戶建立緊實的肌肉。\n
        你訓練過高素質的運動員和電影明星。""",
    model=model,
)
asistant_id = personal_trainer_assis.id
print(asistant_id)

asst_pqiuJ0u4oY3Kajjd7B26NPm8


In [6]:
# === Thread (uncomment this to create your Thread) ===
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "How do I get started working out to lose fat and build muscles?",
        }
    ]
)
thread_id = thread.id
print(thread_id)

thread_AnjVsMZuurSKEs9ug4S8QKjw


In [15]:
# === Hardcode our ids ===
asistant_id = "asst_pqiuJ0u4oY3Kajjd7B26NPm8"
thread_id = "thread_AnjVsMZuurSKEs9ug4S8QKjw"

In [16]:
# ==== Create a Message ====
message = "一天要吃多少蛋白質才能增加肌肉？"
message = client.beta.threads.messages.create(
    thread_id=thread_id, role="user", content=message
)

In [17]:
# === Run our Assistant ===
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=asistant_id,
    instructions="Please address the user as James Bond",
)

In [18]:
def wait_for_run_completion(client, thread_id, run_id, sleep_interval=5):
    """

    Waits for a run to complete and prints the elapsed time.:param client: The OpenAI client object.
    :param thread_id: The ID of the thread.
    :param run_id: The ID of the run.
    :param sleep_interval: Time in seconds to wait between checks.
    """
    while True:
        try:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if run.completed_at:
                elapsed_time = run.completed_at - run.created_at
                formatted_elapsed_time = time.strftime(
                    "%H:%M:%S", time.gmtime(elapsed_time)
                )
                print(f"Run completed in {formatted_elapsed_time}")
                logging.info(f"Run completed in {formatted_elapsed_time}")
                # Get messages here once Run is completed!
                messages = client.beta.threads.messages.list(thread_id=thread_id)
                last_message = messages.data[0]
                response = last_message.content[0].text.value
                print(f"Assistant Response: {response}")
                break
        except Exception as e:
            logging.error(f"An error occurred while retrieving the run: {e}")
            break
        logging.info("Waiting for run to complete...")
        time.sleep(sleep_interval)

In [19]:
# === Run ===
wait_for_run_completion(client=client, thread_id=thread_id, run_id=run.id)

Run completed in 00:00:14
Assistant Response: 嗨，James Bond。增加肌肉需要足夠的蛋白質攝入量，但具體的需求量會因個人的體重、活動水平和目標而有所不同。普遍建議每公斤體重攝取每天1.6-2.2克的蛋白質。例如，如果你的體重是70公斤，那麼每天攝取大約112-154克蛋白質。這些蛋白質可以從各種來源獲得，例如肉類、雞蛋、魚類、奶製品、豆類和堅果等。此外，分散攝取蛋白質是重要的，可以在每餐中均勻分配，以確保肌肉得到持續的養分供應。請記住，諮詢營養專家或體能教練可能有助於制定適合你的具體計劃。祝你成功！


In [20]:
# ==== Steps --- Logs ==
run_steps = client.beta.threads.runs.steps.list(thread_id=thread_id, run_id=run.id)
print(f"Steps---> {run_steps.data[0]}")

Steps---> RunStep(id='step_yX4osRjTOjSeYZ9yw5ig59rs', assistant_id='asst_pqiuJ0u4oY3Kajjd7B26NPm8', cancelled_at=None, completed_at=1709559732, created_at=1709559719, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_kuCTxT7PMEJLXBhQKIzh6ylh', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_YebhCkPM0m6ulYA7dYaivqW7'), type='message_creation'), thread_id='thread_AnjVsMZuurSKEs9ug4S8QKjw', type='message_creation', usage=Usage(completion_tokens=322, prompt_tokens=167, total_tokens=489), expires_at=None)
